In [1]:
import pandas as pd



url = 'https://drive.google.com/uc?id=1eqFIZCJTkXY-PsTeS-uz_DousIH3XQu-'
df = pd.read_csv(url) # Открываем rides_info - информация про поездки
df.sample(5)

,user_id,car_id,ride_id,ride_date,rating,ride_duration,ride_cost,speed_avg,speed_max,stop_times,distance,refueling,user_ride_quality,deviation_normal
556859,v10778590v,n-5592412g,m1P,2020-02-01,1.75,74,586,43,86.000000,2,2911.528874,0,5.232256,17.994
734572,o89864017N,z20306091J,l1H,2020-03-03,7.49,51,709,49,66.163731,0,2400.466154,0,-18.740065,23.679
101552,F16300421C,H-2882173Z,t1t,2020-02-28,2.67,47,509,30,44.022017,0,571.165652,0,-31.958227,2.806
482956,d41272546S,i-1711813H,J1H,2020-02-26,5.05,94,934,54,109.000000,10,750.442517,0,12.982971,3.403
57126,U10907749M,D85554326y,W1D,2020-01-29,4.45,18,282,69,101.372087,0,1234.708189,0,-3.991645,-24.908


> Задание: Создайте в df новый столбец final_ride_cost, который будет зависеть от рейтинга (rating) и максимальной скорости (speed_max) поездки. Значения в этом столбце будут отличаться в зависимости от двух условий:

1) Если рейтинг выше 6 и максимальная скорость поездки меньше 120 км/ч, то стоимость поездки (ride_cost) уменьшите на 5%.

2) В противном случае: стоимость поездки (ride_cost) увеличьте на 5%.

* В новом столбце final_ride_cost отбросьте дробную часть значений, оставьте только целую часть. Так как стоимость измеряется в рублях, дробная часть не имеет значения.

In [ ]:
def help_fun(df):
  if df['rating'] > 6 and df['speed_max'] < 120:
    return int(df['ride_cost'] / 100 * 95)
  else:
    return int(df['ride_cost'] / 100 * 105)

df['new_feature'] = df.apply(help_fun, axis=1)
df.head()

,user_id,car_id,ride_id,ride_date,rating,ride_duration,ride_cost,speed_avg,speed_max,stop_times,distance,refueling,user_ride_quality,deviation_normal,new_feature
0,o52317055h,A-1049127W,b1v,2020-01-01,4.95,21,268,36,113.548538,0,514.246920,0,1.115260,2.909,281
1,H41298704y,A-1049127W,T1U,2020-01-01,6.91,8,59,36,93.000000,1,197.520662,0,1.650465,4.133,56
2,v88009926E,A-1049127W,g1p,2020-01-02,6.01,20,315,61,81.959675,0,1276.328206,0,2.599112,2.461,299
3,t14229455i,A-1049127W,S1c,2020-01-02,0.26,19,205,32,128.000000,0,535.680831,0,3.216255,0.909,215
4,W17067612E,A-1049127W,X1b,2020-01-03,1.21,56,554,38,90.000000,1,1729.143367,0,2.716550,-1.822,581


> Создайте новый датафрейм, в котором для каждой машины есть следующие столбцы:

- Столбец feature_1 - Количество уникальных значений остановок (stop_times).
- Столбец feature_2 - Второй максимум столбца максимальной скорости (speed_max).
- Столбец feature_3 - Отношение минимального и максимального значения рейтинга.
- Столбец feature_4 - Самое популярное значение числа остановок (другим словами, мода).
- Столбец feature_5 - Стандартное отклонение стоимостей поездок (ride_cost).

Решение запишите в переменную result

In [ ]:
func_1 = lambda x: x.nunique()  # Количество уникальных значений остановок
func_2 = lambda x: sorted(x.unique(), reverse=True)[1]  # Второй максимум максимальной скорости
func_3 = lambda x: min(x) / max(x)  # Отношение минимального и максимальному рейтинга
func_4 = lambda x: pd.Series.mode(x)  # Самое часто значение числа остановок. (Мода)

result = df.groupby("car_id", as_index=False).agg(
    feature_1=("stop_times", func_1),
    feature_2=("speed_max", func_2),
    feature_3=("rating", func_3),
    feature_4=("stop_times", func_4),
    feature_5=("ride_cost", 'std'),
)
result.head()

,car_id,feature_1,feature_2,feature_3,feature_4,feature_5
0,A-1049127W,4,176.253340,0.010493,0,59874.910968
1,A-1079539w,5,178.380584,0.010194,0,96142.307393
2,A-1162143G,20,160.879082,0.010929,0,29293.805768
3,A-1228282M,4,179.665429,0.010684,0,100938.229321
4,A-1339912r,5,150.260140,0.010030,0,68850.936082


> Задача: Датасет rides содержит столбцы: car_id, ride_id, user_id, rating, ride_duration, ride_cost

* Обогатите датасет rides про поездки дополнительными признаками без использования метода merge.
* 🤖 Добавьте столбец feature_1 — 75% перцентиль столбца ride_cost для каждого пользователя (user_id).
* 🎃 Добавьте столбец feature_2 — разницу максимального и минимального значения rating, деленную на среднее столбца rating для каждого пользователя (user_id).
* 👽 Добавьте столбец feature_3  — количество значений ride_duration, которое больше 40 для каждой машины (car_id).
* 👾 Добавьте столбец feature_4 — сумму значений ride_duration, которая больше 40 для каждой машины (car_id).
Результат запишите в переменную result.

In [ ]:
rides = pd.read_csv('https://stepik.org/media/attachments/lesson/779916/3_1_6.csv')

In [ ]:
import numpy as np



f1 = lambda x: np.percentile(x, 75)
f2 = lambda x: (max(x) - min(x)) / np.mean(x)
f3 = lambda x: sum(x > 40)
f4 = lambda x: sum(x[x > 40])

rides['feature_1'] = rides.groupby('user_id', as_index=False)['ride_cost'].transform(f1)
rides['feature_2'] = rides.groupby('user_id', as_index=False)['rating'].transform(f2)
rides['feature_3'] = rides.groupby('car_id', as_index=False)['ride_duration'].transform(f3)
rides['feature_4'] = rides.groupby('car_id', as_index=False)['ride_duration'].transform(f4)
#result = rides
rides.head()

,car_id,ride_id,user_id,rating,ride_duration,ride_cost,true_feature_1,true_feature_2,true_feature_3,true_feature_4,feature_1,feature_2,feature_3,feature_4
0,v13590850L,m1q,S54951221d,1.38,52,723,599.25,1.043328,1,52,599.25,1.043328,1,52
1,C-1092574s,q1w,h90416721m,8.36,33572,470001,470001.00,0.000000,6,100329,470001.00,0.000000,6,100329
2,i77678921h,i1i,A15424124M,7.03,49,385,385.00,0.000000,3,169,385.00,0.000000,3,169
3,U16614632F,p1m,Q22613814X,6.87,43,510,510.00,0.000000,5,320,510.00,0.000000,5,320
4,i55328849i,I1A,N22883211D,4.30,110,1536,1536.00,0.000000,10,23108,1536.00,0.000000,10,23108


In [ ]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/779916/3_1_7.csv')

> Задача. На основе этого датафрейма надо будет собрать несколько сводных таблиц (pivot table) и потом смержить их в одну итоговую, получив некое подобие эмбеддингов для подачи в модель.

- Индексом во всех таблицах будет user_id.
- 🙂 Первая таблица: Сколько раз пользователь брал машины определенного класса?
- 🙃 Вторая таблица: Какими были максимальная, минимальная и средняя дистанция и стоимость поездки пользователя?
- 🙂 Третья таблица: Сколько уникальных машин брал пользователь в аренду?
- Далее соединяем по ключу (смерживаем) все 3 таблицы в одну.
- И в конце добавляем бинарный таргет в столбец crazy_driver: Является ли наш пользователь безумным гонщиком? Если его максимальная скорость превышала 100 км/ч, то 1; если не превышала, то 0.
- Результат решения запишите в переменную result.

In [ ]:
df.head()

,user_id,car_type,car_id,distance,ride_cost,speed_max,model
0,h14343699V,economy,A-1049127W,158.842938,135,72.027288,Hyundai Solaris
1,b13580442h,standart,A-1079539w,3288.823484,510,124.758500,Renault Kaptur
2,R21123881y,economy,A-1162143G,1464.013497,584,47.000000,VW Polo
3,P16628111o,standart,A-1079539w,1104.886251,1148,78.926452,Renault Kaptur
4,W15317251O,economy,A-1162143G,983.088780,337,180.000000,VW Polo


In [ ]:
f1 = lambda x: len(set(x))

tabel_1 = df.pivot_table(index = 'user_id',
                                        columns = ['car_type'],
                                        values = ['car_id'],
                                        aggfunc = ['count']).fillna(0)
tabel_1.columns = [f'{i[2]}_{i[0]}' for i in tabel_1.columns]
tabel_2 = df.pivot_table(index = 'user_id',
                                        columns = [],
                                        values = ['distance', 'ride_cost'],
                                        aggfunc = ['max', 'min', 'count']).fillna(0)
tabel_2.columns = [f'{i[1]}_{i[0]}' for i in tabel_2.columns]
tabel_3 = df.pivot_table(index = 'user_id',
                                        columns = [],
                                        values = ['car_id'],
                                        aggfunc = ['nunique']).fillna(0)
tabel_3.columns = [f'{i[1]}_{i[0]}' for i in tabel_3.columns]
tabel_4 = df.pivot_table(index = 'user_id',
                                        columns = [],
                                        values = ['speed_max'],
                                        aggfunc = [lambda x: 1 if max(x) > 100 else 0]).fillna(-1)
tabel_4.columns = [f'{i[1]}_{i[0]}' for i in tabel_4.columns]
res = tabel_1.merge(tabel_2, on = 'user_id', how = 'left')
res = res.merge(tabel_3, on = 'user_id', how = 'left')
res = res.merge(tabel_4, on = 'user_id', how = 'left')
res = res.rename(columns={'speed_max_<lambda>': 'crazy_driver'})
res = res.reset_index()
res.head()
# result =

,user_id,economy_count,standart_count,distance_max,ride_cost_max,distance_min,ride_cost_min,distance_count,ride_cost_count,car_id_nunique,crazy_driver
0,A11173530w,1.0,0.0,2.967598e+03,614,2.967598e+03,614,1,1,1,1
1,A66187474K,0.0,1.0,4.621773e+03,1320,4.621773e+03,1320,1,1,1,0
2,C84438702e,1.0,0.0,1.221070e+03,195,1.221070e+03,195,1,1,1,1
3,E15036556k,0.0,1.0,1.284249e+06,374394,1.284249e+06,374394,1,1,1,0
4,G84350611k,0.0,1.0,1.850353e+03,428,1.850353e+03,428,1,1,1,0


In [2]:
tabel_2 = df.pivot_table(index = 'user_id',
                                        columns = [],
                                        values = ['distance', 'ride_cost'],
                                        aggfunc = ['max', 'min', 'count']).fillna(0)
tabel_2

max                   min              count          
                distance ride_cost    distance ride_cost distance ride_cost
user_id                                                                    
A10036845F  1.800982e+06    303214   70.747299        44       47        47
A10070382r  2.511784e+06    559084   74.689784        17       29        29
A10189856o  2.129678e+06    506227  113.844168        93       39        39
A10302790R  8.747656e+05    274199   71.868624        35       58        58
A10371879j  1.415827e+06    364034   90.030929        86       30        30
...                  ...       ...         ...       ...      ...       ...
z97886080x  1.472806e+06    506697   91.782784        27       61        61
z98024494F  1.349330e+06    372122  218.779586        99       22        22
z98672480L  2.472038e+06    473899  216.073265        50       42        42
z98727090R  1.122946e+06    429037   48.009925        35      113       113
z99757546C  7.656645e+05    124989   67.766375        89       35        35

[15153 rows x 6 columns]